# CMPE 549: Bioinformatics - Fall 2024

## Assignment I: Needleman-Wunsch Pairwise Sequence Alignment 

**Deadline: 12.11.2024 - 23:59**

Please describe your work clearly since this notebook is considered as your report.

- **Name: Dağlar Eren Tekşen**
- **Student ID: 2020400111**

## Part 1 - Your Implementation of the Algorithm

In the first part, you will implement the Needleman-Wunsch pairwise sequence alignment algorithm. The algorithm must be implemented with a linear gap penalty. It should take two amino acid sequences, a gap penalty and a scoring matrix as input and should output the maximum alignment score of these sequences and the alignment achieving this maximum score. If there are multiple optimal alignments, the algorithm should output any one of them. For the scoring matrix (match and mismatch), please use the BLOSUM62 scoring matrix.

In [13]:
import blosum as blo
global matrix
matrix = blo.BLOSUM(62)

In [33]:
def my_needleman_wunsch_algorithm_linear_penalty(sequence_1, sequence_2, gap_penalty, scoring_matrix):
    """
    Implements the Needleman-Wunsch pairwise sequence alignment algorithm with a linear gap penalty.

    Parameters:
    ----------
    sequence_1 : str
        The first amino acid sequence to be aligned.
    sequence_2 : str
        The second amino acid sequence to be aligned.
    gap_penalty : int or float
        The penalty score for introducing a gap in the alignment. This penalty is constant for each gap introduced.
    scoring_matrix : list, dict, numpy array, or similar
        A matrix or dictionary containing scores for each possible amino acid pair. The matrix is used to score matches 
        and mismatches between amino acids.

    Returns:
    -------
    tuple
        A tuple containing:
            - alignment_score (float): The maximum alignment score between the two sequences based on the given scoring matrix.
            - alignment (str): A formatted string representation of the optimal alignment, where:
                - Gaps are represented by '-' symbols.
                - Matching amino acids are represented by '|' symbols.
                - Mismatches are represented by '.' symbols.
                

    Notes:
    -----
    - If there are multiple optimal alignments, any one of them may be returned.
    - This function follows a global alignment approach to find the best match across the entire length of the input sequences.
    - The scoring matrix can be of any format (list, dictionary, numpy array, etc.), as long as it allows scoring each pair 
      of amino acids.

    Example:
    --------
    input:
        sequence_1 = "PTTEINS"
        sequence_2 = "PRTWPSEIN"
        gap_penalty = -1
        scoring_matrix = ...
        
    output:
        (
        24.0,
        "target            0 P-T--TEINS  7
                          0 |-|--.|||- 10
        query             0 PRTWPSEIN-  9"
        )
    """
    # Initialize the scoring matrix for alignment, with an empty list to hold the scores and pointers
    matrix_score = [[]]
    
    # Set the initial cell (0,0) with a score of 0 and mark it as the start position "S"
    matrix_score[0].append([0, "S"])  # START
    
    # Fill in the first row with gap penalties for aligning against the first sequence only (insertions)
    for i in range(1, len(sequence_1) + 1):
        matrix_score[0].append([matrix_score[0][i - 1][0] + gap_penalty, "I"])  # INSERTION
    
    # Fill in the first column with gap penalties for aligning against the second sequence only (deletions)
    for j in range(1, len(sequence_2) + 1):
        matrix_score.append([[matrix_score[j - 1][0][0] + gap_penalty, "D"]])  # DELETION
    
    # Fill in the rest of the matrix with scores based on match/mismatch and gap penalties
    for i in range(1, len(sequence_2) + 1):
        for j in range(1, len(sequence_1) + 1):
            # Calculate the score for a match or mismatch by adding the substitution matrix score
            match = matrix_score[i - 1][j - 1][0] + matrix[sequence_1[j - 1]][sequence_2[i - 1]]
            
            # Calculate the score if aligning by deletion (moving down in the matrix)
            delet = matrix_score[i - 1][j][0] + gap_penalty
            
            # Calculate the score if aligning by insertion (moving right in the matrix)
            inst = matrix_score[i][j - 1][0] + gap_penalty
            
            # Choose the highest score from match, deletion, and insertion options
            values = [match, delet, inst]
            max_value = max(values)
            
            # Determine the source of the highest score to track the alignment path
            index_of_max = values.index(max_value)
            where = "blank"
            if index_of_max == 0:
                where = "M"  # Match/Mismatch
            elif index_of_max == 1:
                where = "D"  # Deletion
            elif index_of_max == 2:
                where = "I"  # Insertion
            
            # Add the highest score and its origin to the scoring matrix
            matrix_score[i].append([max_value, where])
    
    # Initialize strings to hold the final alignment results
    target = ""
    alignment = ""
    query = ""
    
    # Start at the bottom-right of the matrix and trace back to build the alignment
    i = len(sequence_2)
    j = len(sequence_1)
    while i >= 1 and j >= 1:
        # If the pointer is "I", add a gap in the target sequence
        if matrix_score[i][j][1] == "I":
            target = sequence_1[j-1] + target
            alignment = "-" + alignment
            query = "-" + query
            j -= 1
        
        # If the pointer is "D", add a gap in the query sequence
        elif matrix_score[i][j][1] == "D":
            target = "-" + target
            alignment = "-" + alignment
            query = sequence_2[i-1] + query
            i -= 1
        
        # If the pointer is "M", add both characters and match/mismatch symbol to alignment
        else:
            target = sequence_1[j-1] + target
            if sequence_1[j-1] == sequence_2[i-1]:
                alignment = "|" + alignment  # Indicate a match with "|"
            else:
                alignment = "." + alignment  # Indicate a mismatch with "."
            query = sequence_2[i-1] + query
            j -= 1
            i -= 1
    
    # Return the final alignment score and the formatted alignment
    return (
        matrix_score[len(sequence_2)][len(sequence_1)][0],  # Alignment score
        f"target            0 {target} {len(sequence_1)}\n"  # Formatted target sequence
        f"                  0 {alignment} {len(alignment)}\n"  # Formatted alignment line
        f"query             0 {query} {len(sequence_2)}\n"  # Formatted query sequence
    )


    #pass  # Algorithm implementation goes here


## Part 2 - Biopython Implementation

For the second part of the assignment, you will use the [Biopython](https://biopython.org/) package to validate your Needleman-Wunsch implementation. In simple terms, you will write a function that calls the Needleman-Wunsch algorithm implemented in the Biopython package. You should use the built-in BLOSUM62 matrix of Biopython.

In [2]:
from Bio import Align

In [3]:
def biopython_needleman_wunsch_algorithm_linear_penalty(sequence_1, sequence_2, gap_penalty):
    """
    Calls the Needleman-Wunsch pairwise sequence alignment algorithm implemented in the Biopython package
    with a linear gap penalty using the BLOSUM62 scoring matrix.

    Parameters:
    ----------
    sequence_1 : str
        The first amino acid sequence to be aligned.
    sequence_2 : str
        The second amino acid sequence to be aligned.
    gap_penalty : int or float
        The penalty score for introducing a gap in the alignment. This penalty is constant for each gap introduced.

    Returns:
    -------
    tuple
        A tuple containing:
            - alignment_score (float): The maximum alignment score between the two sequences using the BLOSUM62 matrix.
            - alignment (str): A formatted string representation of the optimal alignment where:
                - Gaps are represented by '-' symbols.
                - Matching amino acids are represented by '|' symbols.
                - Mismatches are represented by '.' symbols.

    Notes:
    -----
    - If there are multiple optimal alignments, any one of them may be returned.
    - Use the built-in BLOSUM62 matrix of Biopython.
    - For the returned tuple use score field and format() function of Bio.Align.PairwiseAligner().align() object of Biopython.
    
    Example:
    --------
    input:
        sequence_1 = "PTTEINS"
        sequence_2 = "PRTWPSEIN"
        gap_penalty = -1
        
    output:
        (
        24.0,
        "target            0 P-T--TEINS  7
                          0 |-|--.|||- 10
        query             0 PRTWPSEIN-  9"
        )
    """
    # Initialize the pairwise alignment object from Biopython
    aligner = Align.PairwiseAligner()
    aligner.mode = 'global'
    
    # Load the BLOSUM62 substitution matrix, commonly used for protein sequence alignment
    aligner.substitution_matrix = Align.substitution_matrices.load("BLOSUM62")
    
    # Set the gap opening and extension penalties to the same value for a linear gap penalty model
    aligner.open_gap_score = gap_penalty
    aligner.extend_gap_score = gap_penalty
    
    alignments = aligner.align(sequence_1, sequence_2)
    best_alignment = alignments[0]
    alignment_score = best_alignment.score
    
    # Format the alignment into a human-readable string, with matches, mismatches, and gaps
    formatted_alignment = best_alignment.format()
    
    # Return the alignment score and the formatted alignment
    return alignment_score, formatted_alignment

## Part 3 - Use an Online Tool

For the third part, you will find an online tool that applies the Needleman-Wunsch pairwise sequence alignment algorithm. Give a link to the online tool and explain how to use it in 2-3 sentences. 

A commonly used online tool for the Needleman-Wunsch algorithm is **EMBOSS Needle**:

**Link**: [EMBOSS Needle](https://www.ebi.ac.uk/Tools/psa/emboss_needle/)

**How to Use**: 
1. Go to the EMBOSS Needle tool and enter the two sequences you want to align in the provided text boxes (in FASTA format or plain text).
2. Choose a scoring matrix (like BLOSUM62) and set the gap penalties if needed.
3. Click **Submit** to run the alignment. The output will show the aligned sequences, with scores and indicators for matches, mismatches, and gaps.

## Extra

You can use the functions below to generate random sample sequences to test your algorithms. 

In [6]:
import random

aminoacids = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]


def generate_sequence(length=50):
    return "".join([random.choice(aminoacids) for i in range(length)])


def mutate_sequence(seq, n_mutations=10):
    seq = list(seq)
    pos = {
        random.randint(1, len(seq)): random.choice(["substitute", "delete"])
        for i in range(n_mutations)
    }
    mutated_sequence = ""
    for ix, aminoacid in enumerate(seq):
        if ix in pos:
            if pos[ix] == "substitute":
                mutated_sequence += random.choice(aminoacids)
        else:
            mutated_sequence += aminoacid
    return mutated_sequence

In [37]:
# Generate a random sample sequence
sample_sequence = generate_sequence()

# Create two mutated versions of the sample sequence for alignment testing
sequence1 = mutate_sequence(sample_sequence)
sequence2 = mutate_sequence(sample_sequence)

# My Needleman-Wunsch implementation
# the alignment score (index 0 of the returned tuple)
# the formatted alignment (index 1 of the returned tuple)
print("my function:\n", 
      my_needleman_wunsch_algorithm_linear_penalty(sequence1, sequence2, -1, [])[0], 
      "\n", 
      my_needleman_wunsch_algorithm_linear_penalty(sequence1, sequence2, -1, [])[1])

# Biopython Needleman-Wunsch implementation
# the alignment score (index 0 of the returned tuple)
# the formatted alignment (index 1 of the returned tuple)
print("biopython function:\n", 
      biopython_needleman_wunsch_algorithm_linear_penalty(sequence1, sequence2, -1)[0], 
      "\n", 
      biopython_needleman_wunsch_algorithm_linear_penalty(sequence1, sequence2, -1)[1])


my function:
 197.0 
 target            0 AGMHQYNQKL-CSQIDTCKASTSCIIMYDTYKHNGNV-PFMQVEG-WCP 46
                  0 |||-||||.|-|||||||||.||.|||..|||.||-|-||||||--||| 49
query             0 AGM-QYNQNLTCSQIDTCKANTSLIIMTQTYKWNG-VYPFMQVE-YWCP 46

biopython function:
 197.0 
 target            0 AGMHQYNQKL-CSQIDTCKASTSCIIMY-DTYKH-NGNV-PFMQVEG-WCP 46
                  0 |||-||||.|-|||||||||.||.|||--.|||--||-|-||||||--||| 51
query             0 AGM-QYNQNLTCSQIDTCKANTSLIIM-TQTYK-WNG-VYPFMQVE-YWCP 46

